# Recomendação Música

# 0.0 Imports

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## 0.1. Funções auxiliares

## 0.2 Carregar dados

In [70]:
df_dados = pd.read_csv('../dados/Dados_totais.csv')
df_anos = pd.read_csv('../dados/data_by_year.csv')
df_generos = pd.read_csv('../dados/data_by_genres.csv')

# 1.0. Descrição dos Dados

In [71]:
df1 = df_dados.copy()

In [72]:
df1.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,artists_song
0,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,0,3AJwUDP919kvQ9QcozQPxg,0.000121,11,0.2340,-7.227,1,Yellow,84,0.0281,173.372,Coldplay - Yellow
1,0.613,2000,0.14300,OutKast,0.843,270507,0.806,1,0I3q5fE6wg7LIfHGngUTnV,0.000000,4,0.0771,-5.946,0,Ms. Jackson,80,0.2690,94.948,OutKast - Ms. Jackson
2,0.400,2000,0.00958,Linkin Park,0.556,216880,0.864,0,60a0Rd6pjrkxjPbaKzXjfq,0.000000,3,0.2090,-5.870,0,In the End,84,0.0584,105.143,Linkin Park - In the End
3,0.543,2000,0.00664,3 Doors Down,0.545,233933,0.865,0,6ZOBP3NvffbU4SZcrnt1k6,0.000011,11,0.1680,-5.708,0,Kryptonite,78,0.0286,99.009,3 Doors Down - Kryptonite
4,0.760,2000,0.03020,Eminem,0.949,284200,0.661,1,3yfqSUWxFvZELEM4PmlwIR,0.000000,5,0.0454,-4.244,0,The Real Slim Shady,80,0.0572,104.504,Eminem - The Real Slim Shady


In [73]:
# df_dados.describe()

In [74]:
# df_generos.head()

## 1.1. Dimensão dados

In [75]:
print( 'Número de Linhas: {}'.format(df1.shape[0] ) )
print( 'Número de Colunas: {}'.format(df1.shape[1] ) )

Número de Linhas: 20311
Número de Colunas: 19


## 1.2. Tipo Dados

In [76]:
df1.dtypes

valence             float64
year                  int64
acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
speechiness         float64
tempo               float64
artists_song         object
dtype: object

## 1.4. Validando NA

In [77]:
df1.isna().sum()

valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
speechiness         0
tempo               0
artists_song        0
dtype: int64

## 1.5. Estatística Descritiva

In [78]:
num_attributes = df1.select_dtypes( include=['int64', 'int32', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'int32', 'float64', 'datetime64[ns]'] )

In [79]:
# Tendencia central - mean, ]median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersão - std, min, max, range, knew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply( np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

#Concatenar
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns =['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,valence,0.0,1.000,1.000,0.481928,0.472000,0.246407,0.140008,-0.904303
1,year,2000.0,2020.000,20.000,2010.183201,2010.000000,6.063818,-0.020193,-1.198748
2,acousticness,0.0,0.996,0.996,0.242799,0.114000,0.284409,1.167440,0.161336
3,danceability,0.0,0.986,0.986,0.587702,0.596000,0.171318,-0.362448,-0.119937
4,duration_ms,30301.0,2558287.000,2527986.000,228628.975826,221627.000000,70463.498581,5.492197,120.046653
5,energy,0.0,0.999,0.999,0.650766,0.680000,0.221769,-0.683219,-0.008898
6,explicit,0.0,1.000,1.000,0.279504,0.000000,0.448755,0.982774,-1.034257
7,instrumentalness,0.0,1.000,1.000,0.060173,0.000002,0.199310,3.533004,11.259340
8,key,0.0,11.000,11.000,5.239673,5.000000,3.583391,0.024003,-1.308581
9,liveness,0.0,0.997,0.997,0.194004,0.128000,0.157754,2.169330,5.442713


In [80]:
cat_attributes.apply( lambda x: x.unique().shape[0] )

artists           875
id              20311
name            18691
artists_song    20311
dtype: int64

In [81]:
df_anos['year'].value_counts(normalize=True)

1921    0.01
1984    0.01
1994    0.01
1993    0.01
1992    0.01
        ... 
1951    0.01
1950    0.01
1949    0.01
1948    0.01
2020    0.01
Name: year, Length: 100, dtype: float64

# Feature Engeneering

# Filtragem Variáveis

## Seleção Linhas

In [82]:
df_dados['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [83]:
df_anos['year'].unique

<bound method Series.unique of 0     1921
1     1922
2     1923
3     1924
4     1925
      ... 
95    2016
96    2017
97    2018
98    2019
99    2020
Name: year, Length: 100, dtype: int64>

In [84]:
df_anos = df_anos[df_anos['year'] >= 2000].reset_index(drop=True)

## Seleção Colunas

In [85]:
# removendo algumas features que no primeiro momento não agregam.
df_dados = df_dados.drop(['explicit', 'key', 'mode'], axis=1)
df_generos = df_generos.drop(['key', 'mode'], axis=1)
df_anos = df_anos.drop(['key', 'mode'], axis=1)

In [86]:
df_dados.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,name,popularity,speechiness,tempo,artists_song
0,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,3AJwUDP919kvQ9QcozQPxg,0.000121,0.2340,-7.227,Yellow,84,0.0281,173.372,Coldplay - Yellow
1,0.613,2000,0.14300,OutKast,0.843,270507,0.806,0I3q5fE6wg7LIfHGngUTnV,0.000000,0.0771,-5.946,Ms. Jackson,80,0.2690,94.948,OutKast - Ms. Jackson
2,0.400,2000,0.00958,Linkin Park,0.556,216880,0.864,60a0Rd6pjrkxjPbaKzXjfq,0.000000,0.2090,-5.870,In the End,84,0.0584,105.143,Linkin Park - In the End
3,0.543,2000,0.00664,3 Doors Down,0.545,233933,0.865,6ZOBP3NvffbU4SZcrnt1k6,0.000011,0.1680,-5.708,Kryptonite,78,0.0286,99.009,3 Doors Down - Kryptonite
4,0.760,2000,0.03020,Eminem,0.949,284200,0.661,3yfqSUWxFvZELEM4PmlwIR,0.000000,0.0454,-4.244,The Real Slim Shady,80,0.0572,104.504,Eminem - The Real Slim Shady


# Análise Exploratória

In [87]:
fig = px.line(df_anos, x='year', y='loudness', markers=True, title='Variação do loudness através dos Anos')
fig.show()

In [89]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['acousticness'], name='Acousticness'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['valence'], name='Valence'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['danceability'], name='Danceability'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['energy'], name='Energy'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['instrumentalness'], name='Instrumentalness'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['liveness'], name='Liveness'))
fig.add_trace(go.Scatter(x=df_anos['year'], y=df_anos['speechiness'], name='Speechiness'))

fig.show()

In [111]:
fig = px.imshow(df_dados.corr(), text_auto=True)
fig.show()

C:\Users\laurivan\AppData\Local\Temp\ipykernel_10276\408586802.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



# Preparação dos Dados

## Pipeline

Aplicando StandardScaler e PCA usando Pipeline

In [91]:
SEED = 1234
pca_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('PCA', PCA(n_components=2, random_state=SEED))])

In [92]:
df_generos2 = df_generos.drop('genres', axis=1)
df_generos2.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333
1,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000
2,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000
3,0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882
4,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071


In [93]:
generos_embedding_pca = pca_pipeline.fit_transform(df_generos2)
projecao = pd.DataFrame(generos_embedding_pca, columns=['x', 'y'])
projecao.head()

,x,y
0,5.910268,-0.011146
1,2.787093,4.498483
2,-0.757538,-1.258495
3,1.020521,-0.931690
4,0.753911,-0.999861


# Seleção de Features

Neste ciclo serão usadas todas as features para obter o baseline.

# Machine Learning

In [94]:
kmeans = KMeans(n_clusters=5, random_state=SEED)
kmeans.fit(projecao)
df_generos['cluster'] = kmeans.predict(projecao)
projecao['cluster'] = df_generos['cluster']

In [97]:
df_generos.head()

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,cluster
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,2
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,2
2,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,0
3,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,3
4,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,3


In [99]:
projecao['generos'] = df_generos['genres']
projecao.head()

,x,y,cluster,generos
0,5.910268,-0.011146,2,21st century classical
1,2.787093,4.498483,2,432hz
2,-0.757538,-1.258495,0,8-bit
3,1.020521,-0.931690,3,[]
4,0.753911,-0.999861,3,a cappella


In [103]:
fig = px.scatter(projecao, x='x', y='y', color='cluster', hover_data=['x', 'y', 'generos'])
fig.update_layout( autosize=False, width=800, height=600,)
fig.show()

Explicabilidade das features pelo PCA.

In [107]:
pca_pipeline[1].explained_variance_ratio_.sum()

0.49270669947266443

In [109]:
pca_pipeline[1].explained_variance_.sum()

5.421597305805701

# Fine Tuning parameter

# Modelo Final